# Module 5: Simulating Robots in Gazebo

## 🎯 Learning Objectives
- Understand Gazebo simulation vs RViz visualization
- Adapt URDF for Gazebo with inertial and collision properties
- Spawn robots in Gazebo simulator
- Control robots using Gazebo plugins and bridges
- Create complete simulation packages

---

## 📹 Video Introduction

Watch this video before starting the module:

<div style="padding:56.25% 0 0 0;position:relative;">
  <iframe src="https://player.vimeo.com/video/YOUR_VIDEO_ID" style="position:absolute;top:0;left:0;width:100%;height:100%;" frameborder="0" allow="autoplay; fullscreen; picture-in-picture" allowfullscreen></iframe>
</div>

**Note:** Replace `YOUR_VIDEO_ID` with your actual Vimeo ID.

---

## 🏗️ Gazebo vs RViz: Understanding the Difference

### RViz: Visualization Tool
- **What it does**: Visualizes existing ROS data (URDF, TFs, topics)
- **Limitation**: No simulation, no physics, just display
- **Use case**: Development and debugging

### Gazebo: Simulation Engine
- **What it does**: Simulates physics (gravity, friction, collisions)
- **Advantage**: Test robots without hardware, create custom environments
- **Use case**: Full robot simulation before real-world deployment

![Gazebo Interface](images/gazebo.pdf-image-000.jpg)

**Key Insight**: Use Gazebo for simulation, RViz for visualization. They work together!

---

## ⚙️ Technical Requirements

---

## 🔄 Gazebo-ROS 2 Architecture

### How Gazebo Works with ROS 2
Gazebo runs independently but can connect via `ros_gz_bridge`:

![Gazebo-ROS Bridge](images/gazebo.pdf-image-001.png)

### Key Components:
1. **Gazebo Simulator**: Physics engine with plugins
2. **ROS 2 Nodes**: Your robot application
3. **Bridge**: Translates between Gazebo and ROS 2 topics

**Important**: Gazebo topics ≠ ROS 2 topics. They need bridging!

---

## 📐 Adapting URDF for Gazebo

URDF needs two additions for Gazebo:

### 1. Inertial Properties (`<inertial>` tag)
Each physical link needs mass and inertia matrix. Validate in RViz:

![Validating Inertia in RViz](images/gazebo.pdf-image-004.jpg)

**Add to `common_properties.xacro`:**
```xml
<!-- Box inertia macro -->
<xacro:macro name="box_inertia" params="m x y z o_xyz o_rpy">
    <inertial>
        <mass value="${m}" />
        <origin xyz="${o_xyz}" rpy="${o_rpy}" />
        <inertia ixx="${(m/12) * (z*z + y*y)}" ixy="0" ixz="0"
                 iyy="${(m/12) * (x*x + z*z)}" iyz="0"
                 izz="${(m/12) * (x*x + y*y)}" />
    </inertial>
</xacro:macro>

<!-- Cylinder inertia macro -->
<xacro:macro name="cylinder_inertia" params="m r l o_xyz o_rpy">
    <inertial>
        <mass value="${m}" />
        <origin xyz="${o_xyz}" rpy="${o_rpy}" />
        <inertia ixx="${(m/12) * (3*r*r + l*l)}" ixy="0" ixz="0"
                 iyy="${(m/12) * (3*r*r + l*l)}" iyz="0"
                 izz="${(m/2) * (r*r)}" />
    </inertial>
</xacro:macro>
```

**Apply to links in `mobile_base.xacro`:**
```xml
<!-- For base_link -->
<xacro:box_inertia m="5.0" x="${base_length}" y="${base_width}" z="${base_height}"
                   o_xyz="0 0 ${base_height / 2.0}" o_rpy="0 0 0"/>

<!-- For wheels -->
<xacro:cylinder_inertia m="1.0" r="${wheel_radius}" l="${wheel_length}"
                        o_xyz="0 0 0" o_rpy="${pi / 2.0} 0 0"/>
```

### 2. Collision Properties (`<collision>` tag)
Simpler shapes for collision detection:

![Validating Collision in RViz](images/gazebo.pdf-image-005.png)

**Add to links:**
```xml
<!-- For base_link -->
<collision>
    <geometry>
        <box size="${base_length} ${base_width} ${base_height}" />
    </geometry>
    <origin xyz="0 0 ${base_height / 2.0}" rpy="0 0 0" />
</collision>

<!-- For wheels (use sphere to reduce friction) -->
<collision>
    <geometry>
        <sphere radius="${wheel_radius}" />
    </geometry>
    <origin xyz="0 0 0" rpy="0 0 0" />
</collision>
```

### Validate in RViz:
Check inertia and collision properties in RViz to ensure they're correct!

---

## 🤖 Spawning Robot in Gazebo

### Manual Terminal Commands:

**Terminal 1 - robot_state_publisher:**
```bash
ros2 run robot_state_publisher robot_state_publisher \
  --ros-args -p robot_description:="$(xacro /path/to/robot.urdf.xacro)"
```

**Terminal 2 - Start Gazebo:**
```bash
ros2 launch ros_gz_sim gz_sim.launch.py gz_args:="empty.sdf -r"
```

**Terminal 3 - Spawn Robot:**
```bash
ros2 run ros_gz_sim create -topic robot_description
```

**Result:** Your robot appears in Gazebo!

![Robot in Gazebo](images/gazebo.pdf-image-002.png)

---

## 🎮 Controlling Robot with Gazebo Plugins

### Create Gazebo Xacro File:
`mobile_base_gazebo.xacro`:

```xml
<?xml version="1.0"?>
<robot xmlns:xacro="http://www.ros.org/wiki/xacro">
  
  <!-- Reduce caster wheel friction -->
  <gazebo reference="caster_wheel_link">
    <mu1 value="0.1" />
    <mu2 value="0.1" />
  </gazebo>
  
  <!-- Differential Drive Controller -->
  <gazebo>
    <plugin filename="gz-sim-diff-drive-system"
            name="gz::sim::systems::DiffDrive">
      <left_joint>base_left_wheel_joint</left_joint>
      <right_joint>base_right_wheel_joint</right_joint>
      <frame_id>odom</frame_id>
      <child_frame_id>base_footprint</child_frame_id>
      <wheel_separation>0.45</wheel_separation>
      <wheel_radius>0.1</wheel_radius>
    </plugin>
  </gazebo>
  
  <!-- Joint State Publisher -->
  <gazebo>
    <plugin filename="gz-sim-joint-state-publisher-system"
            name="gz::sim::systems::JointStatePublisher">
    </plugin>
  </gazebo>
  
</robot>
```

**Include in main URDF:**
```xml
<xacro:include filename="$(find my_robot_description)/urdf/mobile_base_gazebo.xacro" />
```

---

## 🌉 Bridging Gazebo-ROS Communications

### Create Bridge Configuration:
`gazebo_bridge.yaml`:

```yaml
- ros_topic_name: "/cmd_vel"
  gz_topic_name: "/model/my_robot/cmd_vel"
  ros_type_name: "geometry_msgs/msg/Twist"
  gz_type_name: "gz.msgs.Twist"
  direction: ROS_TO_GZ

- ros_topic_name: "/joint_states"
  gz_topic_name: "/world/empty/model/my_robot/joint_state"
  ros_type_name: "sensor_msgs/msg/JointState"
  gz_type_name: "gz.msgs.Model"
  direction: GZ_TO_ROS

- ros_topic_name: "/tf"
  gz_topic_name: "/model/my_robot/tf"
  ros_type_name: "tf2_msgs/msg/TFMessage"
  gz_type_name: "gz.msgs.Pose_V"
  direction: GZ_TO_ROS
```

### Start Bridge in Launch File:
```xml
<node pkg="ros_gz_bridge" exec="parameter_bridge">
  <param name="config_file" value="$(var gazebo_config_path)" />
</node>
```

---

## 📦 Complete Simulation Package

### Create Bringup Package:
```bash
cd ~/my_robot_ws/src
ros2 pkg create my_robot_bringup --build-type ament_cmake
cd my_robot_bringup
rm -r include src
mkdir launch config
```

### Package Structure:
```
my_robot_bringup/
├── CMakeLists.txt
├── package.xml
├── launch/
│   └── my_robot.launch.xml
└── config/
    └── gazebo_bridge.yaml
```

### Final Launch File:
```xml
<launch>
  <!-- URDF Path -->
  <let name="urdf_path" 
       value="$(find-pkg-share my_robot_description)/urdf/my_robot.urdf.xacro" />
  
  <!-- Bridge Config -->
  <let name="gazebo_config" 
       value="$(find-pkg-share my_robot_bringup)/config/gazebo_bridge.yaml" />
  
  <!-- RViz Config -->
  <let name="rviz_config" 
       value="$(find-pkg-share my_robot_description)/rviz/urdf_config.rviz" />
  
  <!-- robot_state_publisher -->
  <node pkg="robot_state_publisher" exec="robot_state_publisher">
    <param name="robot_description" 
           value="$(command 'xacro $(var urdf_path)')" />
  </node>
  
  <!-- Start Gazebo -->
  <include file="$(find-pkg-share ros_gz_sim)/launch/gz_sim.launch.py">
    <arg name="gz_args" value="empty.sdf -r" />
  </include>
  
  <!-- Spawn Robot -->
  <node pkg="ros_gz_sim" exec="create" args="-topic robot_description" />
  
  <!-- Gazebo-ROS Bridge -->
  <node pkg="ros_gz_bridge" exec="parameter_bridge">
    <param name="config_file" value="$(var gazebo_config)" />
  </node>
  
  <!-- RViz -->
  <node pkg="rviz2" exec="rviz2" args="-d $(var rviz_config)" />
</launch>
```

---

## 🧪 Testing Your Simulation

### Build and Launch:
```bash
cd ~/my_robot_ws
colcon build --packages-select my_robot_bringup
source install/setup.bash
ros2 launch my_robot_bringup my_robot.launch.xml
```

### Control the Robot:
```bash
# Send movement command
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.5}, angular: {z: 0.0}}"

# Use keyboard control
ros2 run teleop_twist_keyboard teleop_twist_keyboard
```

**Expected Results:**
- Robot moves in Gazebo
- TFs published correctly
- RViz shows robot movement

![Complete Simulation with RViz](images/gazebo.pdf-image-003.png)

---

# Summary

In this module, you learned:

✅ Gazebo is a physics simulator, RViz is a visualization tool

✅ URDF needs `<inertial>` and `<collision>` tags for Gazebo

✅ How to spawn robots in Gazebo with proper launch files

✅ How to add Gazebo plugins for robot control

✅ How to bridge Gazebo-ROS communications

✅ How to create complete simulation packages